In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score

# Define base model class
class BaseModel(nn.Module):
    def __init__(self, transformer_model):
        super(BaseModel, self).__init__()
        self.transformer_model = transformer_model
        self.dropout = nn.Dropout(0.3)

    def forward(self, inputs):
        outputs = self.transformer_model(**inputs).last_hidden_state[:, 0, :]
        outputs = self.dropout(outputs)
        return outputs

# Define stacking model class
class StackingModel(nn.Module):
    def __init__(self, base_model1, base_model2):
        super(StackingModel, self).__init__()
        self.base_model1 = base_model1
        self.base_model2 = base_model2
        self.classifier = nn.Linear(base_model1.transformer_model.config.hidden_size + base_model2.transformer_model.config.hidden_size, 2)

    def forward(self, inputs_bert, inputs_codebert):
        outputs_bert = self.base_model1(inputs_bert)
        outputs_codebert = self.base_model2(inputs_codebert)
        combined = torch.cat((outputs_bert, outputs_codebert), dim=1)
        logits = self.classifier(combined)
        return logits

    def predict(self, sentence1, sentence2):
        self.eval()
        with torch.no_grad():
            inputs_bert = self.base_model1.transformer_model(**sentence1).last_hidden_state[:, 0, :]
            inputs_codebert = self.base_model2.transformer_model(**sentence2).last_hidden_state[:, 0, :]
            combined = torch.cat((inputs_bert, inputs_codebert), dim=1)
            logits = self.classifier(combined)
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            return predicted_class, probabilities.cpu().numpy()

    def trainer(self, train_loader, val_loader, num_epochs=3, learning_rate=2e-5):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer = AdamW(self.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(num_epochs):
            self.train()
            total_loss = 0
            for batch in train_loader:
                inputs_bert, inputs_codebert, labels = batch
                inputs_bert = inputs_bert.to(device)
                inputs_codebert = inputs_codebert.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                logits = self(inputs_bert, inputs_codebert)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}')

        self.evaluate(val_loader)

    def evaluate(self, val_loader):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eval()
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        with torch.no_grad():
            for batch in val_loader:
                inputs_bert, inputs_codebert, labels = batch
                inputs_bert = inputs_bert.to(device)
                inputs_codebert = inputs_codebert.to(device)
                labels = labels.to(device)

                logits = self(inputs_bert, inputs_codebert)
                _, predicted = torch.max(logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        accuracy = correct / total
        precision = precision_score(all_labels, all_predictions)
        recall = recall_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions)

        print(f'Validation Accuracy: {accuracy}')
        print(f'Precision: {precision}')
        print(f'Recall: {recall}')
        print(f'F1-Score: {f1}')

# Example usage
if __name__ == "__main__":
    # Initialize BERT and CodeBERT models and tokenizers
    bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
    bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
    codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
    codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')
    # # Load BERT and CodeBERT models and tokenizers
    # bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
    # bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
    
    # codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
    # codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')
    # Create base models
    base_model1 = BaseModel(bert_model)
    base_model2 = BaseModel(codebert_model)

    # Create stacking model
    stacking_model = StackingModel(base_model1, base_model2)

    # Example training and evaluation
    # Replace train_loader and val_loader with your actual data loaders
    train_loader = []  # Replace with your training data loader
    val_loader = []    # Replace with your validation data loader
    stacking_model.trainer(train_loader, val_loader)
